# HDFD - Weekly Segmented Model X Days - Testing
Testing Weekly Segmented Models (WSMs) for X (e.g., 07, 14, etc.) Days

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

from scipy import stats
get_ipython().magic(u'config IPCompleter.greedy=True')


## Connect with the Database

In [ ]:
from contextlib import closing


import psycopg2
import simplejson
import sys
reload(sys)
sys.setdefaultencoding('utf8')

DEFAULT_DB = 'database'
DEFAULT_HOST = 'database-host'
DEFAULT_PORT = 1234


class PsycopgConnector:
    '''
    A database connector that uses Psycopg to connect to Redshift.

    How to play:

        psy_conn = PsycopgConnector(username, password)
        df = psy_conn.run_query(sql=sql, return_data=True)

    NOTE: This class commits queries to redshift if return_data=False.
    This means INSERT, DROP, TRUNCATE, etc. all work against the DB.
    '''

    def __init__(
        self,
        username=None,
        password=None,
        db=DEFAULT_DB,
        host=DEFAULT_HOST,
        port=DEFAULT_PORT,
    ):

        self.db = DEFAULT_DB
        self.host = DEFAULT_HOST
        self.port = DEFAULT_PORT

        self.username = username
        self.password = password

    def _get_connection(self):

        self.conn = psycopg2.connect(
            dbname=self.db,
            user=self.username,
            password=self.password,
            host=self.host,
            port=self.port
        )

        return self.conn

    def run_query(self, sql, return_data=False):

        with closing(self._get_connection()) as conn:
            with conn, conn.cursor() as cur:
                if return_data:
                    return pd.read_sql(sql=sql, con=conn)
                else:
                    cur.execute(sql)
                    

# Read the database's credentials file 
with open("credential.json.nogit") as fh:
    creds = simplejson.loads(fh.read())
    
username = creds.get("user_name")
password = creds.get("password")

pig = PsycopgConnector(username, password)

## Functions

In [ ]:
# Word count fuction
import re
def words_count (strg):
    
    #print(strg)
    
    if strg == '' or pd.isnull(strg):
        no_of_words = 0
        #print('NaN')
    else:
        strg_words_list = re.findall(r"[\w']+", strg)
        no_of_words = len(strg_words_list)
    return no_of_words   

# 1. Import Invoice Data & Extract Avg Word Counts Features

## 1.01 Invoice within X days

In [ ]:
# SQL for impoorting all invoices created within X days after signup_date
sql_invoices_Xdays_all_accounts = '''invoice_creates_in_last_X_day'''

# Import as dataframe from redshift
df_invoices_Xdays_all_accounts = pig.run_query(sql_invoices_Xdays_all_accounts, return_data=True)

# Words count in different invoice features with textual data
df_invoices_Xdays_all_accounts['avg_wc'] = df_invoices_Xdays_all_accounts.apply(lambda x: words_count(x['invoice_description']), axis=1)
                                                                                                                   
# Filters the text columns from the dataframe
df_invoices_Xdays_all_accounts_fil = df_invoices_Xdays_all_accounts.filter(['features_list'])  
                                                                                                                   
# Summing (grouping) all invoices for a 'id'
df_word_count_Xdays_all_accounts_total = df_invoices_Xdays_all_accounts_fil.groupby('id').mean()  

# Final word count table
df_word_count_Xdays_all_accounts_final = df_word_count_Xdays_all_accounts_total.filter(['features_list'])

# 2. Invoice's Important Features


In [ ]:
################# Invoice Details Features ###############

# SQL query 
sql_rs_invoices_clients_activities_all_accounts = '''sql query for details invoice feature'''

# Import as dataframe from redshift
df_rs_invoices_clients_activities_all_accounts = pig.run_query(sql_rs_invoices_clients_activities_all_accounts, return_data=True)


# 4. Import and Exract Features from Events Data
## 4.1 Event data collection 

In [ ]:
############################### Event Features Extraction ################################

#SQL for events 
sql_events = '''invoice event details'''

# Import as dataframe from redshift
df_events_all_accounts = pig.run_query(sql_events, return_data=True)


## 4.2 Removing whitespce from the event strings

In [ ]:
# Removing row if there is 'None' the event cell
df_events_all_accounts = df_events_all_accounts[~df_events_all_accounts.astype(str).eq('None').any(1)]

# Replace the 'NaN' cell by zero
df_events_all_accounts.fillna(0, inplace=True)

# Using lambda function to remove the white space in the event string name
df_events_all_accounts['event_name'] = df_events_all_accounts.apply(lambda x: x['event'].replace(' ', '').replace('-','').replace('/', ''), axis=1)

# Filtered the events columns for day X
df_events_all_accounts_day_X = df_events_all_accounts[['id', 'event_count_day_X', 'event_name']]

## 4.2 Pivote the events (each unique event become a column)

In [ ]:
### Pivote the Day X Events (Each Unique Event Become a Column)###

# Pivot table based on the unique column value in 'event_name'
df_events_all_accounts_day_X = df_events_all_accounts_day_X.pivot_table(values='event_count_day_X', columns='event_name', index='id', aggfunc=np.sum,  fill_value=0)

# Drop the old column name
df_events_all_accounts_day_X.columns.name = None

# Reset the index
df_events_all_accounts_day_X = df_events_all_accounts_day_X.reset_index()

# Replace 'NaN' with zero
df_events_all_accounts_day_X.fillna(0, inplace=True)

# 5. Merging Invoice Features

In [ ]:
# Merging report system and events data for day X period
df_rs_events_day_X = pd.merge(df_rs_invoices_clients_activities_all_accounts, df_events_all_accounts_day_X,
                             on='id', how='left')

# Merging average word count with 'df_rs_events_day_X'
df_rs_events_avg_wc_day_X = pd.merge(df_rs_events_day_X, df_word_count_Xdays_all_accounts_final,
                                    on='id', how='left')

# 6. Filtering out Platform's test accounts 

In [ ]:
################# Filtering Out Test Accounts #############################################################
test_emails = pd.read_csv("file_path.csv", sep="\t")
test_email_list = list(test_emails['email'])


# Function: Filtering  test account by using admin email
from difflib import SequenceMatcher

def email_match(em, email_list):
    
    L = len(email_list)
    match_score = 0
    
    for i in range(0, L):
        if pd.isnull(em):
            match_score = 0
            break;
        else: 
            match_score =  max(match_score, SequenceMatcher(None,em, email_list[i]).ratio())

    return match_score


# Filtering final data from the  Test emails
df_rs_events_avg_wc_day_X_notest = df_rs_events_avg_wc_day_X[
    df_rs_events_avg_wc_day_X.apply(lambda x: email_match(x['admin_email'], test_email_list) < 0.95, axis=1)]

# 7. Filtering only important features: Day X

In [ ]:
# Importing importing features list
important_features = pd.read_csv("import_feature_list_path", sep="\n,")

# Get the important feature as a list
imp_features_list = list(important_features['important_feature'])

# Adding missing important feature column with zero values (if there any!)
for i in range(len(imp_features_list)):
    if imp_features_list[i] in df_rs_events_avg_wc_day_X_notest.columns:
#         print("True")
        continue;
        
    else:
        print("False: ", imp_features_list[i])
        df_rs_events_avg_wc_day_X_notest[imp_features_list[i]] = 0

# Filtering only important features 
df_imp_features_new_accounts_day_X =\
            df_rs_events_avg_wc_day_X_notest[df_rs_events_avg_wc_day_X_notest.columns.intersection(imp_features_list)]

df_imp_features_new_accounts_day_X = df_imp_features_new_accounts_day_X.reindex(
    sorted(df_imp_features_new_accounts_day_X.columns), axis=1)

# Drop rows with nan value
df_imp_features_new_accounts_day_X = df_imp_features_new_accounts_day_X.dropna()

# 8. Filtering inactive users' accounts

In [ ]:
ex_cols_list = ['feature-list']
cols_list = list(df_imp_features_new_accounts_day_X) 
cols = list(set(cols_list) - set(ex_cols_list))

# Function for aggregating selected column values
def cell_value_sum (row, cols):
    sum = 0
    for i in cols:
        sum = sum + row[i]
    return sum

# Fltering out all inactive users accounts
df_final_features_new_accounts_day_X =\
        df_imp_features_new_accounts_day_X[df_imp_features_new_accounts_day_X.apply(lambda x: cell_value_sum(x, cols) > 0, axis=1)]

df_final_features_new_accounts_day_X.head()

# 9. Saving the filtered features data for new accounts

In [ ]:
# Export filtered features data fro new accounts
today = str(date.today())
path = "saving-path" + today + ".tsv"
df_final_features_new_accounts_day_X.to_csv(path, sep="\t", index=False)